In [1]:
import numpy as np 
import pandas as pd

In [2]:
#For offline installtion this bitsandbytes wheel to be uploaded
!pip install bitsandbytes --no-index --find-links=file:///kaggle/input/bitsandbytes2/bitsandbytes-0.43.0-py3-none-manylinux_2_24_x86_64.whl


Looking in links: file:///kaggle/input/bitsandbytes2/bitsandbytes-0.43.0-py3-none-manylinux_2_24_x86_64.whl
Processing /kaggle/input/bitsandbytes2/bitsandbytes-0.43.0-py3-none-manylinux_2_24_x86_64.whl


In [3]:
!pip install accelerate

In [4]:
!pip install peft --no-index --find-links=file:///kaggle/input/peftfile2/peft-0.10.0-py3-none-any.whl

Looking in links: file:///kaggle/input/peftfile2/peft-0.10.0-py3-none-any.whl
Processing /kaggle/input/peftfile2/peft-0.10.0-py3-none-any.whl


In [5]:
#loading the base model in 4 bit
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
bnb_config = BitsAndBytesConfig(
    
    load_in_4bit=True,bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/gemma/transformers/7b-it/3")
model = AutoModelForCausalLM.from_pretrained("/kaggle/input/gemma/transformers/7b-it/3",quantization_config=bnb_config, device_map={"":0})

Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [6]:
#Attaching the adapter files
peft_model_id = "/kaggle/input/adaptermodel2/"
model.load_adapter(peft_model_id)

In [8]:
#Poost processing to remove any unwanted sequence of words from original and rewritten text. 
import string
import re
def new_original(original,text):
    answer=text
    listt=["Hello world"]
    try:
        words = original.split()
        for i in range(0,len(words)):
            if len(words) % 2 == 0:
                if i == len(words)-3:
                    strr = words[i+1]+" "+words[i+2]
                    strr = strr.strip()
                    if strr in answer:
                        listt.append(strr)
                        return listt
            else:
                if i == len(words)-3:
                    strr = words[i] +" "+words[i+1]+" "+words[i+2]
                    strr = strr.strip()
                    if strr in answer:
                        listt.append(strr)
                        return listt
            strr = words[i] + " "+words[i+1]
            strr = strr.strip()
            if strr in answer:
                listt.append(strr)
            
        return listt
    except:
        
        return listt
def new_rewritten(rewritten,text):    
    answer=text
    listt=["Hello world"]
    try:
        words = rewritten.split()
        for i in range(0,len(words)):
            if len(words) % 2 == 0:
                if i == len(words)-3:
                    strr = words[i+1]+" "+words[i+2]
                    strr = strr.strip()
                    if strr in answer:
                        listt.append(strr)
                        return listt
                    else:
                        return listt
            else:
                if i == len(words)-3:
                    strr = words[i] +" "+words[i+1]+" "+words[i+2]
                    strr = strr.strip()
                    if strr in answer:
                        listt.append(strr)
                        return listt
                    else:
                        return listt
            strr = words[i] + " "+words[i+1]
            strr = strr.strip()
            if strr in answer:
                listt.append(strr)
        return listt
    except:
        
        return listt
    
def remove_punctuation(text):
    try:
        pattern = r'[{}]'.format(re.escape(string.punctuation+"."))
        cleaned_text = re.sub(pattern, ' ', text)
        return cleaned_text
    except:
        return text


def remove_double_spaces(text):
    try:
        cleaned_text = re.sub(r'\s+', ' ', text)
        return cleaned_text
    except:
        return text

In [9]:
import re
import time
test = pd.read_csv("/kaggle/input/llm-prompt-recovery/test.csv")
test.reset_index(inplace=True,drop=True)
for index,value in enumerate(test['original_text']):
    
    original = value
    rewritten = test.loc[index,'rewritten_text']
    
    
        
    
    
        
    prompt = f"""<bos><start_of_turn>user
Hello Need Help . My Friend translated my original text into something new .\nOriginal Text : {original}\nRewritten Text : {rewritten}\nMy friend used a large language model to rewrite this . 
Can you craft a exact single-line prompt which he might have used against my original text to get to the rewritten text.<end_of_turn>
<start_of_turn>model Prompt:"""
    
    inputs = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")
    outputs = model.generate(input_ids=inputs.to(model.device),max_new_tokens=100,temperature=0.7,do_sample=True)
    answer = tokenizer.decode(outputs[0])
    
    try:
        
        start = answer.find("<start_of_turn>model")
        answer = answer[start+20:]
        answer = answer.replace("Prompt:","")
        answer = answer.replace("<eos>","")
    
        answer = remove_punctuation(answer)
        original = remove_punctuation(original)
        rewritten = remove_punctuation(rewritten)
        
        answer = answer.lower()
        original = original.lower()
        rewritten = rewritten.lower()
        
        answer = remove_double_spaces(answer)
        original = remove_double_spaces(original)
        rewritten = remove_double_spaces(rewritten)
        
        
        finalans = answer
        try:
            list1 = new_original(original,answer)
            list2 = new_rewritten(rewritten,answer)
            if len(list2) > len(list1):
                if len(list2) != 0:
                    for words in list2:
                        
                        answer = answer.replace(words,"")
                
            elif len(list2) < len(list1):
                if len(list1) != 0:
                    for words in list1:
                        
                        answer = answer.replace(words,"")
                
                
                
        except:
            answer = finalans
        answer = answer.replace(".","")
        answer = answer.replace("*","")
        answer = answer.replace(":","")
        answer = answer.replace(",","")
        answer = answer.replace("using a large language model","")
        answer = answer.replace("additional requirements","")
        answer = answer.replace("additional instructions","")
        answer = answer.replace("translated","rewritten")
        answer = answer.replace("style of writing"," ")
        answer = answer.replace("use of large language models","")
        answer = answer.replace("large language models","")
        answer = answer.replace("using a large language model","")
        answer = answer.replace("large language model","")
        answer = answer.replace("emphasizing","")
        answer = answer.replace("revise","reframe")
        answer = answer.replace("please ensure that the translation is accurate and captures the essence of the original text","")
        answer = ' '.join([word for word in answer.split() if len(word) > 1]) 
        if len(answer) == 0:
            answer = """Please Improve,Rewrite and Rephrase the provided text in a style that adopts the characteristics of the original text, while enhancing the clarity, elegance, and impact. Maintain the core meaning and factual accuracy of the original, and utilize figurative language - including rhyme and playful language where appropriate - creatively to achieve the desired effect aligning with the tone, diction, and stylistic elements of the original text. """
        answer = answer.strip()   
        #mean prompt + llm response
        test.loc[index,'rewrite_prompt'] = "Please improve this text using the writing style with maintaining the original meaning but altering the tone reimagining it through the lens "+answer
    except:
        answer = tokenizer.decode(outputs[0])
        start = answer.find("<start_of_turn>model")
        answer = answer[start+20:]
        answer = answer.lower()
        answer = answer.replace("prompt:","")
        answer = answer.replace("<eos>","")
        answer = answer.replace("using a large language model","")
        answer = answer.replace("additional requirements","")
        answer = answer.replace("additional instructions","")
        answer = answer.replace("translated","rewritten")
        answer = answer.replace("use of large language models","")
        answer = answer.replace("large language models","")
        answer = answer.replace("using a large language model","")
        answer = answer.replace("large language model","")
        answer = answer.replace("revise","reframe")
        #mean prompt + llm response
        test.loc[index,'rewrite_prompt'] = "Please improve this text using the writing style with maintaining the original meaning but altering the tone reimagining it through the lens "+answer

        
        
test = test[['id','rewrite_prompt']]
test.to_csv("submission.csv",index=False)

2024-04-11 22:42:21.454900: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-11 22:42:21.455027: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-11 22:42:21.737297: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
print(test.loc[0,'rewrite_prompt'])

Please improve this text using the writing style with maintaining the original meaning but altering the tone reimagining it through the lens convert this into shanty
